# Summary

Selecting subsets in  [pandas](https://pandas.pydata.org/pandas-docs/stable/) is easy, but the syntax could be cleaner. This post contains two useful monkey patches to pandas to make subset selection great again.

### Here is the code

In [1]:
import pandas as pd

def subset(d, predicate, *args, **kwargs):
    """
    Get a subset of a pandas NDFrame using a predicate.
    
    Equivalent to d[predicate(d, *args, **kwargs)]
    """
    return d[predicate(d, *args, **kwargs)]

# Monkey-patch the NDFrame to allow DataFrame.subset(predicate)
pd.core.generic.NDFrame.subset = subset

def groupby_subset(groupby, predicate, *args, **kwargs):
    """
    Apply subsetting for each group in a pandas GroupBy.
    
    Roughly equivalent to 
    """
    result = groupby.apply(subset, predicate, *args, **kwargs)
    
    any_group_index = next(iter(groupby.groups.values()))
    orig_n_levels = len(any_group_index.levels)
    result_n_levels = len(result.index.levels)
    levels_to_drop = list(range(result_n_levels - orig_n_levels))
    
    return result.droplevel(levels_to_drop)

# And monkey-patch GroupBy to allow GroupBy.subset(predicate)
pd.core.groupby.GroupBy.subset = groupby_subset

### And this is what it enables

In [2]:
data = pd.read_csv('data.csv', index_col=['A', 'B'])
data

a     b   c
A B                
X 1  -4.0  -3.0  -2
  2   NaN  -1.0   0
  3   1.0   2.0   3
Y 2   NaN   4.0   5
  3   6.0   NaN   7
  4   8.0   9.0  10
  5  11.0  12.0  13

In [3]:
(
    data
    .subset(lambda d: d.c > 0)
    .subset(lambda d: d.b <= 9)
)

a    b   c
A B              
X 3  1.0  2.0   3
Y 2  NaN  4.0   5
  4  8.0  9.0  10

In [4]:
(
    data
    .groupby('A')
    .subset(lambda group: group.c < group.c.median())
)

a    b  c
A B             
X 1 -4.0 -3.0 -2
Y 2  NaN  4.0  5
  3  6.0  NaN  7

# Motivation

When working with `DataFrame`s and `Series` I often want to select subsets of the data depending on its values, e.g., select all rows where some column is within certain bounds. This is slighly inconvenient. One possible way is as follows:

In [5]:
selected = data
selected = selected[selected.c > 0]
selected = selected[selected.b <= 9]
selected

a    b   c
A B              
X 3  1.0  2.0   3
Y 2  NaN  4.0   5
  4  8.0  9.0  10

This works but it is not beautiful: the intermediate `selected` needs to be passed into each step. Lots of repetition.

Another option that I usually prefer is to chain calls like this:

In [6]:
selected = (
    data
    .pipe(lambda d: d[d.c > 0])
    .pipe(lambda d: d[d.b <= 9])
)

This is better but it is not great. On each row we are repeating `pipe` which says nothing about what is going on, and we are repeating the subset selection `d[...]` on each row.

The functions defined above give a better, cleaner syntax which clearly tells the reader that subset selection is what is going on:

In [7]:
selected = (
    data
    .subset(lambda d: d.c > 0)
    .subset(lambda d: d.b <= 9)    
)

# It allows predicate with extra arguments

This is a silly example, but the idea is quite useful in some cases.

In [8]:
def complicated_predicate(d, colname, target_value=0, abstol=0):
    return (d[colname] - target_value).abs() <= abstol

(
    data
    .subset(complicated_predicate, 'a', target_value=1, abstol=5)
    .subset(complicated_predicate, 'c', abstol=3)
)

a    b  c
A B             
X 1 -4.0 -3.0 -2
  3  1.0  2.0  3

# It also works for Series

In [9]:
(
    data.c
    .subset(lambda s: s > 2)
)

A  B
X  3     3
Y  2     5
   3     7
   4    10
   5    13
Name: c, dtype: int64

# Using the `subset` method on `GroupBy`

This will give the same result as applying the subsetting on each group separately and then concatenating a new object from the subsets. Note that the resulting index has the same levels as the original data.

In [10]:
# With a DataFrame
(
    data
    .groupby(['A'])
    .subset(lambda d: d.c > d.c.median())
)

a     b   c
A B                
X 3   1.0   2.0   3
Y 4   8.0   9.0  10
  5  11.0  12.0  13

In [11]:
# And also works with a Series
(
    data['c']
    .groupby(['A'])
    .subset(lambda s: s > s.median())
)

A  B
X  3     3
Y  4    10
   5    13
Name: c, dtype: int64

# Using `DataFrame.eval()`

It is possible to make a small addition to allow the predicate to be a string, in which case it can be evaluated using `DataFrame.eval()`, like so:


In [12]:
def subset_with_eval(d, predicate, *args, **kwargs):
    if isinstance(predicate, str):
        return d[d.eval(predicate)]
    return d[predicate(d, *args, **kwargs)]

# Monkey-patch the NDFrame to allow DataFrame.subset(predicate)
pd.core.generic.NDFrame.subset = subset_with_eval

In [13]:
(
    data
    .subset('c > 0')
    .subset('b <= 9')
)

a    b   c
A B              
X 3  1.0  2.0   3
Y 2  NaN  4.0   5
  4  8.0  9.0  10